In [3]:
import pandas as pd
temp_df=pd.read_excel('STOP NUMBERING AND CENTROID NUMBERING.xlsx')
temp_df.to_csv('stops_centroid_numbering.csv')

In [2]:

stops_df=pd.read_csv('remaining_stops_numbering.csv')
stops_df = stops_df.drop(columns=['Unnamed: 0'])
stops_df.head()

,STOP.NO,STOP NAME,STOP LAT,STOP LONG,DIRECTION,CENTROID NUMBERING
0,10001.0,High Court,13.085009,80.283447,Down,1.0
1,10002.0,Flower Bazzar,13.087507,80.281419,Up,2.0
2,10003.0,Broadway,13.088206,80.283890,Down,3.0
3,10003.0,Broadway,13.088206,80.283890,Up,3.0
4,10004.0,Flower Bazzar,13.088298,80.281341,Up,2.0


In [28]:
ticket_df=pd.read_csv('./Final_ETM_Data/Final_Ticket_data_2024-07-01.csv')
ticket_df.columns

Index(['id', 'depot_code', 'depot', 'fleet_no', 'route_service_type',
       'trip_no', 'ticket_no', 'date', 'time', 'ticket_type', 'source',
       'source_stop_no', 'source_zonal_centroid_number', 'destination',
       'destination_stop_no', 'destination_zonal_centroid_number',
       'passenger_count', 'luggage_count', 'passenger_amount',
       'luggage_amount', 'driver_no', 'conductor_no', 'waybill_no',
       'schedule_no', 'payment_type', 'direction', 'Trip_Origin',
       'Trip_Destination', 'total_ticket_amount', 'KM', 'duty_date',
       'Direction_route'],
      dtype='object')

In [29]:
stops_df = stops_df.drop_duplicates(subset='Stop Name')

# Merge ticket_df with stops_df to fill missing source_stop_no
ticket_df = ticket_df.merge(
    stops_df[['STOP.NO', 'Stop Name']],
    how='left',
    left_on='source',
    right_on='Stop Name'
).rename(columns={'STOP.NO': 'source_stop_no_temp'})

# Merge again for missing destination_stop_no
ticket_df = ticket_df.merge(
    stops_df[['STOP.NO', 'Stop Name']],
    how='left',
    left_on='destination',
    right_on='Stop Name'
).rename(columns={'STOP.NO': 'destination_stop_no_temp'})

# Update missing values for source_stop_no and destination_stop_no
ticket_df['source_stop_no'] = ticket_df['source_stop_no'].fillna(ticket_df['source_stop_no_temp'])
ticket_df['destination_stop_no'] = ticket_df['destination_stop_no'].fillna(ticket_df['destination_stop_no_temp'])

# Drop temporary columns used for merging
ticket_df = ticket_df.drop(columns=['source_stop_no_temp', 'destination_stop_no_temp', 'Stop Name_x', 'Stop Name_y'], errors='ignore')

ticket_df.to_csv('Updated_file_with_stops.csv')

In [30]:
ticket_df=pd.read_csv('Updated_file(with stops).csv')
#number of sources in etm file
source_list=ticket_df['source'].str.upper().unique()
print(len(source_list))

1171


In [31]:
destination_list=ticket_df['destination'].str.upper().unique()
print(len(destination_list))

1168


In [32]:
stop_list=stops_df['Stop Name'].unique()
source_arr=[]
source_count=0
for name in source_list:
    if name in stop_list: 
        source_count+=1
    else:
        source_arr.append(name)
        
print(len(source_arr))

917


In [33]:
dest_arr=[]
dest_count=0
for name in destination_list:
    if name in stop_list: 
        dest_count+=1
    else:
        dest_arr.append(name)

print(dest_count)

255
